In [1]:
import boto3
import xmltodict
import json
import pandas as pd

In [2]:
create_hits_in_production = True
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [12]:
# This will return your current MTurk balance if you are connected to Production.
# If you are connected to the Sandbox it will return $10,000.
print(client.get_account_balance()['AvailableBalance'])

21.87


In [4]:
batch_df = pd.read_csv('../../data/batched/batch_type_mturk_pilot.csv', index_col=False)
print(batch_df.shape)
batch_df

(40, 84)


,hearing_id,hearing_title,hearing_date,hearing_summary,turn1_id,q1_speaker,q1_speaker_detail,q1_text,r1_id,r1_speaker,...,r5_speaker_detail,r5_text,prev_turn5_id,prev_turn5_speaker,prev_turn5_speaker_detail,prev_turn5_text,next_turn5_id,next_turn5_speaker,next_turn5_speaker_detail,next_turn5_text
0,CHRG-116hhrg35589,The State Department's Foreign Policy Strategy...,3/27/19,Secretary of State Mike Pompeo testified befor...,1,Chairman Engel,"politician: ELIOT L. ENGEL, New York, Chairman","Can you tell us, then, Mr. Secretary, why have...",2,Secretary Pompeo,...,"witness: Pompeo, Hon. Michael, Secretary, Uni...",Yes.,15,Chairman Engel,"politician: ELIOT L. ENGEL, New York, Chairman","Thank you, Mr. McCaul. We will now go to Mr...",18,Mr. Sires,"politician: ALBIO SIRES, New Jersey","Do you know, can you share with us kind of san..."
1,CHRG-115hhrg32380,Oversight of the Department of Justice,11/14/17,Attorney General Jeff Sessions testified befor...,0,Chairman Goodlatte,"politician: BOB GOODLATTE, Virginia, Chairman","Well, thank you, General Sessions. We will ...",1,Attorney General Sessions,...,witness: The Honorable Jefferson B. Sessions I...,"I do not have the statistics on that, but I be...",9,Attorney General Sessions,witness: The Honorable Jefferson B. Sessions I...,"We have--members of the committee, we had abou...",12,Chairman Goodlatte,"politician: BOB GOODLATTE, Virginia, Chairman","And, finally, as you know, this committee did ..."
2,CHRG-116hhrg35230,"Hearing with Michael Cohen, Former Attorney to...",2/27/19,"Michael Cohen, President Trump’s former lawyer...",0,Chairman Cummings,"politician: ELIJAH E. CUMMINGS, Maryland, Chai...","Thank you very much, Mr. Cohen. I now recogniz...",1,Mr. Cohen,...,"witness: Michael Cohen, Former Attorney to Pr...","Back in 2017 when--actually, I apologize. In 2...",7,Mr. Cohen,"witness: Michael Cohen, Former Attorney to Pr...","The statement is accurate, but the discussions...",10,Chairman Cummings,"politician: ELIJAH E. CUMMINGS, Maryland, Chai...","In your testimony, you said you bought some ch..."
3,CHRG-115hhrg30956,Facebook: Transparency and Use of Consumer Data,4/11/18,Facebook CEO Mark Zuckerberg testified before ...,1,Mr. Walden,"politician: GREG WALDEN, Oregon",All right. Let me ask the next one. You can se...,2,Mr. Zuckerberg,...,"witness: Mark Zuckerberg, Cofounder, Chairman...","Congressman, yes. In response to these issues,...",9,Mr. Zuckerberg,"witness: Mark Zuckerberg, Cofounder, Chairman...","Congressman, yes. We limit a lot of the data t...",12,Mr. Pallone,"politician: FRANK PALLONE, Jr., New Jersey","But, see, again, I don't see that in the chang..."
4,CHRG-114hhrg26003,Holding Wall Street Accountable: Investigating...,9/29/16,Wells Fargo Chair and CEO John Stumpf appeared...,0,Chairman Hensarling,"politician: JEB HENSARLING, Texas, Chairman",The Chair now yields himself 5 minutes for que...,1,Mr. Stumpf,...,"witness: Stumpf, John G., Chairman and Chief ...",People will be reviewed across the board at ho...,7,Mr. Stumpf,"witness: Stumpf, John G., Chairman and Chief ...","I can't give you a specific timeframe, Mr. Cha...",10,Chairman Hensarling,"politician: JEB HENSARLING, Texas, Chairman","Okay, but holding people accountable--isn't it..."
5,CHRG-116hhrg37282,Violations of the Hatch Act Under the Trump Ad...,6/26/19,The House Oversight and Reform Committee held ...,8,Chairman Cummings,"politician: ELIJAH E. CUMMINGS, Maryland, Chai...",What message does the President's refusal to f...,9,Mr. Kerner,...,"witness: The Honorable Henry Kerner, Special ...","No, I don't believe so.",15,Mr. Kerner,"witness: The Honorable Henry Kerner, Special ...",Correct.,18,Chairman Cummings,"politician: ELIJAH E. CUMMINGS, Maryland, Chai...",Are you aware whether President Trump took any...
6,CHRG-116hhrg36621,Hearing with Secretary of Commerce Wilbur L. R...,3/14/19,Commerce Secretary Wilbur Ross testified befor...,2,Chairman Cummings,"politician: ELIJAH E. CUMMINGS, Maryland, Chai...",

In [5]:
html_layout = open('./mturk_hit_type.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

In [6]:
TaskAttributes = {
    'MaxAssignments': 3,     
    # How long the task will be available on the MTurk website (1 week)
    'LifetimeInSeconds': 60*60*24*7, 
    # How long before automatically approving workers (3 days)
    'AutoApprovalDelayInSeconds': 60*60*24*3, 
    # How long Workers have to complete each item (1 hour)
    'AssignmentDurationInSeconds': 60*60, 
    # The reward you will offer Workers for each response
    'Reward': '0.35',                     
    'Title': 'What type of questions and responses are in U.S. congressional hearings?',
    'Keywords': 'dialogue, politics, question, answer, categorization',
    'Description': 'Categorize the type of questions and responses in witness questioning of U.S. congressional hearings.'
}


In [7]:
column_names = list(batch_df.columns)
column_to_xml = {}
for column_name in column_names:
    column_to_xml[column_name] = '${'+column_name+'}'

for k,v in list(column_to_xml.items()):
    if v not in question_xml:
        print('Warning: Ignoring value: ', v)
        column_to_xml.pop(k)

In [8]:
"""
Create hits

"""

results = []
hit_type_id = ''
for hit in batch_df.itertuples():
    # Convert named tuple to dictionary
    question_xml_filled = question_xml
    hit_dict = hit._asdict()
    for key in hit_dict.keys():
        if key in column_to_xml:
            question_xml_filled = question_xml_filled.replace(column_to_xml[key],str(hit_dict[key]))
    response = client.create_hit(
       **TaskAttributes,
       Question=(question_xml_filled),
       #insert quals here
        QualificationRequirements=[
            {
                'QualificationTypeId': '000000000000000000L0',
                'Comparator': 'GreaterThanOrEqualTo',
                'IntegerValues': [
                    95,
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
            {
                'QualificationTypeId': '00000000000000000071',
                'Comparator': 'EqualTo',
                'LocaleValues': [
                    {
                        'Country': 'US'
                    }
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
            {
                'QualificationTypeId': '00000000000000000040',
                'Comparator': 'GreaterThanOrEqualTo',
                'IntegerValues': [
                    500,
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
        ]
        
     )
    hit_type_id = response['HIT']['HITTypeId']
    hit_dict['hit_id'] = response['HIT']['HITId']
    results.append(hit_dict)
print("Created ", len(results), " hits.")   
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

Created  40  hits.
You can view the HITs here:
https://www.mturk.com/mturk/preview?groupId=3ZDEOZ9P08JVWCDZ4FWZNT4HR9PM6B


## Get Results

In [9]:
def get_true(my_dict):
    for key in my_dict.keys():
        if my_dict[key]:
            return key
    return None

In [39]:
data = {}
for item in results: 
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
    )
    
    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)
    answers = []
    
    # register HIT
    data[item['hit_id']] = {}
    data[item['hit_id']]['hearing_id'] = item['hearing_id']
    data[item['hit_id']]['hearing_title'] = item['hearing_title']
    data[item['hit_id']]['qa1'] = {'q_id': item['turn1_id'], 'r_id': item['r1_id'], 'answers': []}
    data[item['hit_id']]['qa2'] = {'q_id': item['turn2_id'], 'r_id': item['r2_id'], 'answers': []}
    data[item['hit_id']]['qa3'] = {'q_id': item['turn3_id'], 'r_id': item['r3_id'], 'answers': []}
    data[item['hit_id']]['qa4'] = {'q_id': item['turn4_id'], 'r_id': item['r4_id'], 'answers': []}
    data[item['hit_id']]['qa5'] = {'q_id': item['turn5_id'], 'r_id': item['r5_id'], 'answers': []}
    
    for assignment in assignments:
        # Retrieve the attributes for each Assignment
        worker_id = assignment['WorkerId']
        work_time_seconds = (assignment['SubmitTime'] - assignment['AcceptTime']).total_seconds()
        assignment_id = assignment['AssignmentId']
        
        # Retrieve the value submitted by the Worker from the XML
        xml_doc = xmltodict.parse(assignment['Answer'])
        answers_dict = json.loads(xml_doc['QuestionFormAnswers']['Answer']['FreeText'])[0]
        answers.append(answers_dict)
        
        feedback = None
        if 'feedback' in answers_dict:
            feedback = answers_dict['feedback']
        
        for idx in range(1,5+1):
            q_type_key = f'questionType{idx}'
            r_type_key = f'responseType{idx}'
            if r_type_key not in answers_dict:
                info = {
                    'assignment_id': assignment_id,
                    'worker_id': worker_id,
                    'work_time_secs': work_time_seconds,
                    'feedback': feedback,
                    'q_type': get_true(answers_dict[q_type_key]),
                    'r_type': None,
                }
            else:
                info = {
                    'assignment_id': assignment_id,
                    'worker_id': worker_id,
                    'work_time_secs': work_time_seconds,
                    'feedback': feedback,
                    'q_type': get_true(answers_dict[q_type_key]),
                    'r_type': get_true(answers_dict[r_type_key]),
                }
            qa_key = f'qa{idx}'
            data[item['hit_id']][qa_key]['answers'].append(info)
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
   
        
        

In [40]:
data

{'3MA5N0ATTCAFABKJGTR7MVIZG5KWKZ': {'hearing_id': 'CHRG-116hhrg35589',
  'hearing_title': "The State Department's Foreign Policy Strategy and FY20 Budget Request",
  'qa1': {'q_id': 1,
   'r_id': 2,
   'answers': [{'assignment_id': '3E1QT0TDFQ8F1ZW78PVQJJWNHEY8IK',
     'worker_id': 'A29VL3MZE7YPBZ',
     'work_time_secs': 644.0,
     'feedback': None,
     'q_type': 'wh',
     'r_type': 'answer'},
    {'assignment_id': '35BLDD71I7WZ71HUGMUMBSJLOQSVZ2',
     'worker_id': 'A3IPVESY1MUTW1',
     'work_time_secs': 221.0,
     'feedback': None,
     'q_type': 'wh',
     'r_type': 'answer'},
    {'assignment_id': '3IAS3U3I0GFUSCGCMYU8N5F70Q6B2Z',
     'worker_id': 'A3HSONMX6LNZR4',
     'work_time_secs': 428.0,
     'feedback': None,
     'q_type': 'wh',
     'r_type': "can't-answer"}]},
  'qa2': {'q_id': 3,
   'r_id': 4,
   'answers': [{'assignment_id': '3E1QT0TDFQ8F1ZW78PVQJJWNHEY8IK',
     'worker_id': 'A29VL3MZE7YPBZ',
     'work_time_secs': 644.0,
     'feedback': None,
     'q_type': 

In [41]:
with open('../../data/annotated/batch_type_mturk_pilot_results.json','w+') as f:
    f.write(json.dumps(data))

In [46]:
reject_workers = ['A2RHE12CJSKCRE', 'A30US88GBGBG0V', 'A2ZX7580NVWA6I', 'A3U53LW0JV9UQJ']

client.reject_assignment(
    AssignmentId='3DUZQ9U6SNN26PVGLU0EFZMI1BWSVY',
    RequesterFeedback='easy items were not marked correctly'
)

{'ResponseMetadata': {'RequestId': '3e6ebe94-9dd6-466e-a148-34d7c7754c33',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3e6ebe94-9dd6-466e-a148-34d7c7754c33',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Thu, 17 Oct 2019 17:26:52 GMT'},
  'RetryAttempts': 0}}

### check assignment status

In [49]:
from datetime import datetime

# get all HITS
hits = []
paginator = client.get_paginator('list_hits')
i=0
for page in paginator.paginate():
    hits.extend(page['HITs'])
    i += 1
    if i > 30:
        break;

for item in hits:
        hit_id=item['HITId']
        
        hit_type_id = item['HITTypeId']
        # Get HIT status
        status=client.get_hit(HITId=hit_id)['HIT']['HITStatus']
        
        if hit_type_id=='3ZDEOZ9P08JVWCDZ4FWZNT4HR9PM6B':
            if status=='Reviewable':
                assignmentsList = client.list_assignments_for_hit(
                    HITId=hit_id)
                assignments = assignmentsList['Assignments']
                for assignment in assignments:
                    ass_id = assignment['AssignmentId']
                    print("assignment id: ", ass_id, ", status: ", assignment['AssignmentStatus'], assignment['SubmitTime'],assignment['AutoApprovalTime'])

assignment id:  3IQ1VMJRYUJ09MWJ1T9R2H3FOC79A2 , status:  Submitted 2019-10-16 09:58:30-05:00 2019-10-19 09:58:30-05:00
assignment id:  3E47SOBEYRVHD2MKP1BVL5S675GCIR , status:  Submitted 2019-10-16 10:02:46-05:00 2019-10-19 10:02:46-05:00
assignment id:  3X0H8UUIT2NFLEVLB45A115W8JZWSB , status:  Submitted 2019-10-16 10:03:17-05:00 2019-10-19 10:03:17-05:00
assignment id:  3KV0LJBBH3K7FN1USDZI2TY9OF3MRN , status:  Submitted 2019-10-16 10:01:07-05:00 2019-10-19 10:01:07-05:00
assignment id:  3VD82FOHKRNR9OXNNWO052OSJFNCOW , status:  Submitted 2019-10-16 10:20:27-05:00 2019-10-19 10:20:27-05:00
assignment id:  32SCWG5HII3KE70NS5OC8218XXY6P3 , status:  Submitted 2019-10-16 10:37:09-05:00 2019-10-19 10:37:09-05:00
assignment id:  379J5II41PFY01E46VE339K19LLELY , status:  Submitted 2019-10-16 09:56:44-05:00 2019-10-19 09:56:44-05:00
assignment id:  3C5W7UE9CGPRCJN61C5ARBKXPWOMXJ , status:  Submitted 2019-10-16 10:03:04-05:00 2019-10-19 10:03:04-05:00
assignment id:  345LHZDEDYRY9TL1QPP8B5CN

assignment id:  3CTOC39K38P7WWBUG2OKHKFSJ9BJ7E , status:  Submitted 2019-10-16 09:59:27-05:00 2019-10-19 09:59:27-05:00
assignment id:  3QIYRE09Y4GP40ND6KZ39H5WP7GN1P , status:  Submitted 2019-10-16 10:02:55-05:00 2019-10-19 10:02:55-05:00
assignment id:  3X66WABAJXHF4BK66K0R7QBWMFD3GU , status:  Submitted 2019-10-16 10:09:03-05:00 2019-10-19 10:09:03-05:00
assignment id:  3KRVW3HTZOKJG21772ZNS335AHOSMJ , status:  Submitted 2019-10-16 10:14:51-05:00 2019-10-19 10:14:51-05:00
assignment id:  3G5W44VEU8HL09SDRQSQOCSYX3CGKZ , status:  Submitted 2019-10-16 12:26:15-05:00 2019-10-19 12:26:15-05:00
assignment id:  3B2X28YI3XEIADZD906BDIWJGVW6BZ , status:  Submitted 2019-10-16 12:40:20-05:00 2019-10-19 12:40:20-05:00
assignment id:  3DEL4X4EL7KR60CKKFTLL674Y6ZYXI , status:  Submitted 2019-10-16 09:55:28-05:00 2019-10-19 09:55:28-05:00
assignment id:  3GD6L00D3TWJMI2DUJK3W2GWKH7M1P , status:  Submitted 2019-10-16 09:59:37-05:00 2019-10-19 09:59:37-05:00
assignment id:  354P56DE9L20VZUK3WXY9L6F

### delete hits

In [ ]:
from datetime import datetime

# get all HITS
hits = []
paginator = client.get_paginator('list_hits')
i=0
for page in paginator.paginate():
    hits.extend(page['HITs'])
    i += 1
    if i > 30:
        break;

# delete
#    for item in client.list_hits()['HITs']:
for item in hits:
        hit_id=item['HITId']
        print('HITId:', hit_id)

        hit_type_id = item['HITTypeId']
        # Get HIT status
        status=client.get_hit(HITId=hit_id)['HIT']['HITStatus']
        print('HITStatus:', status)

        if hit_type_id=='3ZDEOZ9P08JVWCDZ4FWZNT4HR9PM6B':
            # If HIT is active then set it to expire immediately
            if status=='Assignable':
                response = client.update_expiration_for_hit(
                    HITId=hit_id,
                    ExpireAt=datetime(2015, 1, 1)
                )        
            if status=='Reviewable':
                assignmentsList = client.list_assignments_for_hit(
                    HITId=hit_id)
                assignments = assignmentsList['Assignments']
                for assignment in assignments:
                    ass_id = assignment['AssignmentId']
                    client.approve_assignment(AssignmentId=ass_id)
            # Delete the HIT
            try:
                client.delete_hit(HITId=hit_id)
            except:
                print('Not deleted')
            else:
                print('Deleted')